In [1]:
import xarray as xr
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os

Navigate to ftp://ftp.cdc.noaa.gov/Datasets/ncep/
Download a bunch of .nc files (All stored together on one page, choose the ones you want.
Here I use:
air.sfc.2020.nc (Surface air temperature, daily observations in 2020, 2.5 degree spatial resolution)
rhum.sfc.2020.nc (Surface level relative humidity, daily observations in 2020, 2.5 degree spatial resolution)
srfp.2020.nc (Surface level air pressure, daily observations in 2020, 2.5 degree spatial resolution)
uwnd.2020.nc (Meridional wind speed at 13 altitudes, daily 2020, 2.5 degree spatial resolution)
vwnd.2020.nc (Zonal wind speed at 13 altitudes, etc.)

I also want sea surface temperature (SST), which sadly is not included in this large data dump. So, go over to
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198207/

This dataset is nice enough to have separate data files for every day, so it takes some extra work to download them.

For this demo I downloaded October 23, 2020 through October 25, 2020, because there's a tropical depression in the gulf coast that's been brewing.

The file for SST consists of interpolated data, so it has daily observations for temperature, temp anomaly, and mean square error, all at a 0.25 degree spatial resolution.

While all these datasets are worldwide, we really only care about the North Atlantic for the scope of this project. Thus, in pre-processing data, I seek to accomplish a couple things:

Limit the datasets to the relevant region and dates
Combine the datasets so all the data I need is in one place
Choose data from the specific dates/coordinates relevant to the current storm

In [2]:
#Read data and trim it
myfile = xr.open_dataset('rhum.sfc.2020.nc')
print(myfile)
myfile = myfile.where(myfile.lat > 10, drop = True)
myfile = myfile.where(myfile.lat<45, drop = True)
myfile = myfile.where(myfile.lon > 258, drop = True)
myfile = myfile.where(myfile.lon<345, drop = True)

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 300)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-10-26
Data variables:
    rhum     (time, lat, lon) float32 ...
    head     (time) int16 ...
Attributes:
    title:          Once daily NCEP geopotential height data 
    delta_time:     once daily
    supplier:       NCEP
    producer:       NCEP
    history:        created 2019/12 by Hoop (netCDF3)
    description:    Data is from NCEP initialized analysis\n(2x/day).  It con...
    platform:       Model
    Conventions:    CF-1.2
    References:     https://www.psl.noaa.gov/data/gridded/data.ncep.html
    dataset_title:  NCEP Global Data Assimilation System GDAS


C:\Users\mdona\Anaconda3\lib\site-packages\xarray\conventions.py:500: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [3]:
#Now how big is it?
print(myfile)

<xarray.Dataset>
Dimensions:  (lat: 13, lon: 34, time: 300)
Coordinates:
  * lon      (lon) float32 260.0 262.5 265.0 267.5 ... 335.0 337.5 340.0 342.5
  * lat      (lat) float32 42.5 40.0 37.5 35.0 32.5 ... 22.5 20.0 17.5 15.0 12.5
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-10-26
Data variables:
    rhum     (time, lat, lon) float32 68.25 78.240005 ... 81.649994 78.520004
    head     (time, lat, lon) float64 0.0 0.0 0.0 ... -3.277e+04 -3.277e+04
Attributes:
    title:          Once daily NCEP geopotential height data 
    delta_time:     once daily
    supplier:       NCEP
    producer:       NCEP
    history:        created 2019/12 by Hoop (netCDF3)
    description:    Data is from NCEP initialized analysis\n(2x/day).  It con...
    platform:       Model
    Conventions:    CF-1.2
    References:     https://www.psl.noaa.gov/data/gridded/data.ncep.html
    dataset_title:  NCEP Global Data Assimilation System GDAS


In [4]:
#Shaved to relevant dimensions, now let's do dates!
myfile = myfile.where(myfile.time.dt.month == 10, drop = True)
myfile = myfile.where(myfile.time.dt.day > 22, drop = True)
print(myfile)

<xarray.Dataset>
Dimensions:  (lat: 13, lon: 34, time: 4)
Coordinates:
  * lon      (lon) float32 260.0 262.5 265.0 267.5 ... 335.0 337.5 340.0 342.5
  * lat      (lat) float32 42.5 40.0 37.5 35.0 32.5 ... 22.5 20.0 17.5 15.0 12.5
  * time     (time) datetime64[ns] 2020-10-23 2020-10-24 2020-10-25 2020-10-26
Data variables:
    rhum     (time, lat, lon) float32 83.56999 82.25 ... 81.649994 78.520004
    head     (time, lat, lon) float64 -3.277e+04 -3.277e+04 ... -3.277e+04
Attributes:
    title:          Once daily NCEP geopotential height data 
    delta_time:     once daily
    supplier:       NCEP
    producer:       NCEP
    history:        created 2019/12 by Hoop (netCDF3)
    description:    Data is from NCEP initialized analysis\n(2x/day).  It con...
    platform:       Model
    Conventions:    CF-1.2
    References:     https://www.psl.noaa.gov/data/gridded/data.ncep.html
    dataset_title:  NCEP Global Data Assimilation System GDAS


In [5]:
#Great. Now we just need to do this with the other ones and merge them
#We have to treat the wind differently because of altitude levels, so just temp and pressure for now
for i in ['srfp.2020.nc', 'air.sfc.2020.nc']:
    workFile = xr.open_dataset(i)
    workFile = workFile.where(workFile.lat > 10, drop = True)
    workFile = workFile.where(workFile.lat<45, drop = True)
    workFile = workFile.where(workFile.lon > 258, drop = True)
    workFile = workFile.where(workFile.lon<345, drop = True)
    workFile = workFile.where(workFile.time.dt.month == 10, drop = True)
    workFile = workFile.where(workFile.time.dt.day > 22, drop = True)
    myfile = xr.merge([myfile, workFile])
    workFile.close()

C:\Users\mdona\Anaconda3\lib\site-packages\xarray\conventions.py:500: SerializationWarning: variable 'srfp' has multiple fill values {-999.0, -9.96921e+36}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
C:\Users\mdona\Anaconda3\lib\site-packages\xarray\conventions.py:500: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [6]:
print(myfile)

<xarray.Dataset>
Dimensions:  (lat: 13, lon: 34, time: 4)
Coordinates:
  * lon      (lon) float32 260.0 262.5 265.0 267.5 ... 335.0 337.5 340.0 342.5
  * lat      (lat) float32 42.5 40.0 37.5 35.0 32.5 ... 22.5 20.0 17.5 15.0 12.5
  * time     (time) datetime64[ns] 2020-10-23 2020-10-24 2020-10-25 2020-10-26
Data variables:
    rhum     (time, lat, lon) float32 83.56999 82.25 ... 81.649994 78.520004
    head     (time, lat, lon) float64 -3.277e+04 -3.277e+04 ... -3.277e+04
    srfp     (time, lat, lon) float32 936.48737 968.0814 ... 1011.00305
    air      (time, lat, lon) float32 270.72998 272.35 274.0 ... 300.06 300.75


In [8]:
windy = xr.open_dataset('uwnd.2020.nc')
print("Altitude values, in terms of pressure: {}".format(windy.level.values))

Altitude values, in terms of pressure: [1000.  850.  700.  500.  400.  300.  250.  200.  150.  100.   70.   50.]


C:\Users\mdona\Anaconda3\lib\site-packages\xarray\conventions.py:500: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


The first inclination would be to use the surface reading for wind, since that's what we care about in a hurricane. The issue here is that friction between the sea and the air causes funky readings. So, we elect to go with the 850 hPa level.

In [9]:
windy = windy.where(windy.level == 850, drop = True)
windy2 = xr.open_dataset('vwnd.2020.nc')
windy2 = windy2.where(windy2.level == 850, drop = True)

#Then we process/merge the same way

for i in [windy, windy2]:
    i = i.where(i.lat > 10, drop = True)
    i = i.where(i.lat < 45, drop = True)
    i = i.where(i.lon > 258, drop = True)
    i = i.where(i.lon < 345, drop = True)
    i = i.where(i.time.dt.month == 10, drop = True)
    i = i.where(i.time.dt.day > 22, drop = True)
    myfile = xr.merge([myfile, i])
    
windy.close()
windy2.close()

C:\Users\mdona\Anaconda3\lib\site-packages\xarray\conventions.py:500: SerializationWarning: variable 'vwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [10]:
myfile

<xarray.Dataset>
Dimensions:  (lat: 13, level: 1, lon: 34, time: 4)
Coordinates:
  * lon      (lon) float32 260.0 262.5 265.0 267.5 ... 335.0 337.5 340.0 342.5
  * lat      (lat) float32 42.5 40.0 37.5 35.0 32.5 ... 22.5 20.0 17.5 15.0 12.5
  * time     (time) datetime64[ns] 2020-10-23 2020-10-24 2020-10-25 2020-10-26
  * level    (level) float32 850.0
Data variables:
    rhum     (time, lat, lon) float32 83.56999 82.25 ... 81.649994 78.520004
    head     (time, lat, lon) float64 -3.277e+04 -3.277e+04 ... -3.277e+04
    srfp     (time, lat, lon) float32 936.48737 968.0814 ... 1011.00305
    air      (time, lat, lon) float32 270.72998 272.35 274.0 ... 300.06 300.75
    uwnd     (time, level, lat, lon) float32 -0.90000916 ... -6.430008
    vwnd     (time, level, lat, lon) float32 -13.800003 -12.869995 ... -2.449997

In [11]:
#Looks good, now just save this new datafile! It is much smaller.
myfile.to_netcdf('Combined_data.nc')
myfile.close()

In [13]:
#Now onto SST
myfile1 = xr.open_dataset('oisst-avhrr-v02r01.20201023_preliminary.nc')
myfile2 = xr.open_dataset('oisst-avhrr-v02r01.20201024_preliminary.nc')
myfile3 = xr.open_dataset('oisst-avhrr-v02r01.20201025_preliminary.nc')

#What's it look like
myfile1

<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 1, zlev: 1)
Coordinates:
  * time     (time) datetime64[ns] 2020-10-23T12:00:00
  * zlev     (zlev) float32 0.0
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.375 359.625 359.875
Data variables:
    sst      (time, zlev, lat, lon) float32 ...
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.20201023_preliminary.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    history:                    Final file created using preliminary as first...
    date_modified:              2020-10-24T09:02:00Z
    date_created:               2020-10-24T09:02:00Z
    product_version:            Version v02r01
    processing_level:           NOAA Level 4
    institution:                NOAA/National Centers for Environmental Infor...
    creator_url:                https://www.ncei.noaa.gov/
    creator_email:              oisst-help@noaa.gov
    keywords:                   Earth Science > Oceans > Ocean Temperature > ...
    keywords_vocabulary:        Global Change Master Directory (GCMD) Earth S...
    platform:                   Ships, buoys, Argo floats, MetOp-A, MetOp-B
    platform_vocabulary:        Global Change Master Directory (GCMD) Platfor...
    instrument:                 Earth Remote Sensing Instruments > Passive Re...
    instrument_vocabulary:      Global Change Master Directory (GCMD) Instrum...
    standard_name_vocabulary:   CF Standard Name Table (v40, 25 January 2017)
    geospatial_lat_min:         -90.0
    geospatial_lat_max:         90.0
    geospatial_lon_min:         0.0
    geospatial_lon_max:         360.0
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.25
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.25
    time_coverage_start:        2020-10-23T00:00:00Z
    time_coverage_end:          2020-10-23T23:59:59Z
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR

In [14]:
#Just shave coordinates

myfile1 = myfile1.where(myfile1.lat > 10, drop = True)
myfile1 = myfile1.where(myfile1.lat < 45, drop = True)
myfile1 = myfile1.where(myfile1.lon > 258, drop = True)
myfile1 = myfile1.where(myfile1.lon < 345, drop = True)

myfile2 = myfile2.where(myfile2.lat > 10, drop = True)
myfile2 = myfile2.where(myfile2.lat < 45, drop = True)
myfile2 = myfile2.where(myfile2.lon > 258, drop = True)
myfile2 = myfile2.where(myfile2.lon < 345, drop = True)

myfile3 = myfile3.where(myfile3.lat > 10, drop = True)
myfile3 = myfile3.where(myfile3.lat < 45, drop = True)
myfile3 = myfile3.where(myfile3.lon > 258, drop = True)
myfile3 = myfile3.where(myfile3.lon < 345, drop = True)
    
    
savefile = xr.merge([myfile1, myfile2, myfile3])
savefile

<xarray.Dataset>
Dimensions:  (lat: 140, lon: 348, time: 3, zlev: 1)
Coordinates:
  * time     (time) datetime64[ns] 2020-10-23T12:00:00 ... 2020-10-25T12:00:00
  * zlev     (zlev) float32 0.0
  * lat      (lat) float32 10.125 10.375 10.625 10.875 ... 44.375 44.625 44.875
  * lon      (lon) float32 258.125 258.375 258.625 ... 344.375 344.625 344.875
Data variables:
    sst      (time, zlev, lat, lon) float32 28.59 28.58 28.57 ... 16.27 16.21
    anom     (time, zlev, lat, lon) float32 0.75 0.74 0.74 ... -0.07 -0.16 -0.21
    err      (time, zlev, lat, lon) float32 0.12 0.12 0.12 ... 0.19 0.14999999
    ice      (time, zlev, lat, lon) float32 nan nan nan nan ... nan nan nan nan

In [34]:
myfile1.close()
myfile2.close()
myfile3.close()
savefile.to_netcdf('SST_Data.nc')

In [36]:
savefile.close()